# Get config parameters

## Data needed
We need three informations:
- the **ID of the group** library.  
  Can be found by opening the group’s page: https://www.zotero.org/groups/groupname,   
  and hovering over the group settings link.
- the **API key** from the Zotero [site](https://www.zotero.org/settings/keys/new)
- **library_type** 
  - own Zotero library --> user
  - shared library --> group
  
## Config file

Rename `config_template.cfg` to `config.cfg` and populate it with the three information as explained above.

In [ ]:
%run config.ipynb

In [ ]:
print(f"library_id  : {library_id}")
print(f"api_key     : {api_key}")
print(f"library_type: {library_type}")

# Retrieve data from server 

In [ ]:
lib_items = retrieve_data()

# Get Items with duplicate pdf files
<a id='fetch-duplicates'></a>

In [ ]:
print("Resolving duplicates...")
items_duplicate_attach, pdf_attachments = get_items_with_duplicate_pdf(lib_items)
print("done!")

# Report items with multiple attachments

Multiple attachments are ok.  
We are looking for duplicate pdf files.   


In [ ]:
for item in lib_items:
    key = item["key"]
    firstname = "UNKNOWN"
    lastname = "UNKNOWN"
    creators = item["data"]["creators"]  # could be author or editor
    for creator in creators:
        if creator["creatorType"] == "author":
            firstname = creator["firstName"]
            lastname = creator["lastName"]
            break

#     print(
#         f"""
#     Key: {item['data']['key']}
#     Title: {item['data']['title']}
#     Author: {firstname}, {lastname}
#     PDF attachements: {pdf_attachments[key]}
#     ----"""
#     )

print(f"found {len(items_duplicate_attach)} items with duplicate pdf files")

# Check if Trash is empty

In [ ]:
if len(zot.trash()) > 0:
    print("Trash is not empty. Consider emptying it!")

## Remove items with duplicate attachments

**WARNING**: This cell is dangerous!

Here, duplicate attachments are getting removed.

Execution of [Get Items with duplicate pdf files](#fetch-duplicates) is necessary 
to fetch `items_duplicate_attach` and `pdf_attachments`

In [ ]:
print("Updating library...")
print("===========")
# items_duplicate_attach, pdf_attachments = get_items_with_duplicate_pdf(lib_items)
deleted_attachment = False

for item in items_duplicate_attach:
    files = pdf_attachments[item["key"]]
    cs = zot.children(item["key"])
    print("-----")

    # DANGER AREA!!
    if (
        len(set(files)) == 1 and len(files) > 1
    ):  # some items have different pdf files, like suppl materials. Should not delete
        # here attachments are all named the same -->  a sign of duplicates
        print("all files are the same. Proceed deleting ..")
        print(files)
        for child in cs[1:]:
            if attachment_is_pdf(child):
                print(f"delete {child['data']['filename']}")
                zot.delete_item(child)
                deleted_attachment = True

    else:  # manual mode!
        for child in cs[1:]:
            if attachment_is_pdf(child):
                # ask only for pdf files. Other files, like notes, zip, etc, should not be deleted, anyway.
                answer = input(f"delete {child['data']['filename']}? (y[N])")
                if answer == "y":
                    print(f"deleting {child['data']['filename']}")
                    zot.delete_item(child)
                    deleted_attachment = True

print("===========")
T = dt.datetime.now()
if deleted_attachment:
    print("Attachments deleted!")
else:
    print("No attachments deleted!")

print(f"Done at {T.hour}:{T.minute}:{T.second}")

## Delete duplicate tag
<a id='tags'></a>

In [ ]:
if DELETE_TAGS and deleted_attachement:
    zot.delete_tags("#duplicate-citation-key")